In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
import os

class Video_preproc():
	def __init__(self, base_path):
		self.base_path = base_path
		self.videos = os.listdir(base_path)
		print(f"Found {len(self.videos)} videos in the folder")

	def get_info(self, videoname):
		gameKey, playID, view = videoname.split(".")[0].split('_')
		return gameKey, playID, view

	def count_frames(self, videoname):
		j = 0
		vid = cv2.VideoCapture(self.base_path+videoname)
		while True :
			ret, _ = vid.read()
			if ret == True:
				j+=1
			else:
				break
		return j
	
	def get_boxes(self, fdataframe):

		impact = []
		noimpact = []

		boxes = fdataframe.drop(columns=["gameKey", "playID", "view", "video", "frame","impactType", "label"]).to_numpy()
		boxes = np.nan_to_num(boxes)

		for box in boxes:
			if box[4]==1 and box[5]>0 and box[6]:
				impact.append(list(box[:4]))
			else:
				noimpact.append(list(box[:4]))

		return impact, noimpact	

	def draw_rectangle(self, image, box, color, thickness=3):
		x,w,y,h = box
		image = cv2.rectangle(image, (int(x),int(y)), (int(x+w),int(y+w)), color, thickness)
		return image

	def annotateframe(self, filename, frame, frameno, vdataframe, saveimpactf=False, quality=25):

		files = os.listdir(os.getcwd())
		if saveimpactf and "impactframes" not in files:
			os.system("mkdir impactframes")
		
		fdataframe = vdataframe[vdataframe["frame"]==frameno]
		impact, noimpact = self.get_boxes(fdataframe)
		impact_color = (0,0,255)
		noimpact_color = (0,0,0)
		
		if len(noimpact)>0:
			for box in noimpact:
				frame = self.draw_rectangle(frame, box, noimpact_color, thickness=1)

		if len(impact) > 0:
			for box in impact:
				frame = self.draw_rectangle(frame, box, impact_color, thickness=2)
				if saveimpactf:
					cv2.imwrite(f"impactframes/{filename}", frame,[cv2.IMWRITE_JPEG_QUALITY, quality])

		return frame

	def export_one(
		self, videoname, exportdir,
		quality=25, annotate = False,
		showframes = False, target_shape=None,
		show_progress=False, vdataframe = None,
		saveimpactf=False, save=True
	):
		gameKey, playID, view = self.get_info(videoname)
		
		video = cv2.VideoCapture(self.base_path+videoname)
		i = 1

		if show_progress:
			pbar = tqdm(total=self.count_frames(videoname))

		while(True):
			ret, frame = video.read()

			if ret :
				file_name = f'{exportdir}{gameKey}_{playID}_{view}_{i}.jpeg'
				
				if annotate:
					frame = self.annotateframe(f"{gameKey}_{playID}_{view}_{i}.jpeg",
								frame, i, vdataframe, 
								saveimpactf, quality)
				if target_shape != None:
					frame = cv2.resize(frame, target_shape)

				if showframes:
					pass
					#cv2.imshow(videoname, frame)
					#cv2.waitKey(5)

				if save:
					cv2.imwrite(file_name, frame, [cv2.IMWRITE_JPEG_QUALITY, quality])
				i+=1
			else:
				break

			if show_progress:
				pbar.update(1)

		
	def export(self, exportdir,
			first=None, quality=25, annotate = False,
			target_shape=None, dataframe = None,
			saveimpactf=False, save=True, 
			showframes=False,
		):
		if first != None:
			videos = self.videos[:int(first)]
		else:
			videos = self.videos

		vbar = tqdm(total=len(videos))

		for video in videos:

			if annotate:
				vdf = dataframe[(dataframe["video"]==video)]
				self.export_one(
					video, exportdir, quality=quality,
					annotate=annotate, showframes=showframes,
					target_shape=target_shape, show_progress=False,
					vdataframe=vdf , saveimpactf=saveimpactf,
					save=save
				)
			else:
				self.export_one(
					video, exportdir, quality=quality,
					annotate=annotate, showframes=showframes,
					target_shape=target_shape, show_progress=False,
					vdataframe=None , saveimpactf=saveimpactf,
					save=save
					)
			vbar.update(1)

In [ ]:
! mkdir ./train

video_base_path = r"../input/nfl-impact-detection/train/"
train_label_dic = r"../input/nfl-impact-detection/train_labels.csv"
export_base = r"./train/"

vlabels = pd.read_csv(train_label_dic)

video_proc = Video_preproc(video_base_path)
"""video_proc.export_one(videos_lst[0], export_base, 
                    showframes=True, show_progress=True, 
                    target_shape=None, annotate=True,
                    vdataframe=vlabel, saveimpactf=True, save=True)"""

video_proc.export(export_base,
			first=None, quality=75, annotate = False,
			target_shape=None, dataframe = vlabels,
			saveimpactf=False, save=True, showframes=False)

In [ ]:
lst = os.listdir("./train")
print(len(lst))